# 言語処理100本ノック 2020 第３章

Reference: https://nlp100.github.io/ja/ch03.html

In [5]:
!wget https://nlp100.github.io/data/jawiki-country.json.gz

In [27]:
import json, gzip
import numpy as np

## 20. JSONデータの読み込み

Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [25]:
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        if tmp["title"] == "イギリス": break
tmp["text"][:200]

'{{redirect|UK}}\n{{redirect|英国|春秋時代の諸侯国|英 (春秋)}}\n{{Otheruses|ヨーロッパの国|長崎県・熊本県の郷土料理|いぎりす}}\n{{基礎情報 国\n|略名  =イギリス\n|日本語国名 = グレートブリテン及び北アイルランド連合王国\n|公式国名 = {{lang|en|United Kingdom of Great Britain and Norther'

## 21. カテゴリ名を含む行を抽出

記事中でカテゴリ名を宣言している行を抽出せよ．

In [51]:
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    cat_lines = []
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        lines = tmp["text"].split("\n")
        cat_lines += [line for line in lines if "Category:" in line]
cat_lines[:5]

['[[Category:エジプト|*]]',
 '[[Category:共和国]]',
 '[[Category:軍事政権]]',
 '[[Category:フランコフォニー加盟国]]',
 '{{Main|オーストリア料理|Category:オーストリアの食文化}}']

## 22. カテゴリ名の抽出

記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [102]:
import re
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    categories = []
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        lines = tmp["text"].split("\n")
        categories += [re.split("\\]|\\)|\\|", line.strip("[](){}").split("Category:")[1])[0] for line in lines if "Category:" in line]
np.unique(categories)[:5]

array(['1704年設立', '1801年に成立した国家・領域', '1804年に成立した国家・領域', '1886年に成立した国家・領域',
       '1922年に成立した国家・領域'], dtype='<U32')

## 23. セクション構造

記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [101]:
import re
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    sections = []
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        lines = tmp["text"].split("\n")
        for l in lines:
            if "|section=" in l:
                s_s = l.split("|")
                s_title = s_s[0].strip("{")
                s_level = re.search("section=[0-9]*", l).group().split("=")[1]
                sections.append((s_title,s_level))
sections[:5]

[('独自研究', '1'), ('出典の明記', '1'), ('独自研究', '1'), ('出典の明記', '1'), ('出典の明記', '1')]

## 24. ファイル参照の抽出

記事から参照されているメディアファイルをすべて抜き出せ

In [107]:
import re
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    files = []
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        lines = tmp["text"].split("\n")
        files += [re.search("ファイル:.*", l).group().split(":")[1].split("|")[0].split("]")[0] for l in lines if "ファイル:" in l]
files[:5]

['Coat_of_arms_of_Egypt.svg',
 'All Gizah Pyramids.jpg',
 'Egyptiska hieroglyfer, Nordisk familjebok.png',
 'ModernEgypt, Muhammad Ali by Auguste Couder, BAP 17996.jpg',
 'Gamal Nasser.jpg']

## 25. テンプレートの抽出

記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [162]:
import re
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    result = dict()
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        d = dict()
        ll = re.search("{{基礎情報 (.|\n)+\n}}", tmp["text"]).group().split("\n|")
        for l in ll[1:]: d[l.split("=")[0].strip(" ")] = l.split("=")[1].strip(" ")
        result[tmp["title"]] = d
result

IndexError: list index out of range